# Benchmark building 

# starting by champs Elysées

In [1]:
import pandas as pd
import numpy as np

In [2]:
champ = pd.read_csv('champ.csv',sep=';')
champ.shape

(9338, 15)

In [42]:
champ.head()

,Identifiant arc,Libelle,Date et heure de comptage,Débit horaire,Taux d'occupation,Etat trafic,Identifiant noeud amont,Libelle noeud amont,Identifiant noeud aval,Libelle noeud aval,Etat arc,Date debut dispo data,Date fin dispo data,geo_point_2d,geo_shape
0,4264,AV_Champs_Elysees,2023-02-01T23:00:00+01:00,764.0,11.28000,Fluide,2294,Av_Champs_Elysees-Washington,2293,Av_Champs_Elysees-Berri,Invalide,2005-01-01,2019-06-01,"48.87153587897717, 2.3017227924560615","{""coordinates"": [[2.3009951475338766, 48.87177..."
1,4264,AV_Champs_Elysees,2023-02-01T22:00:00+01:00,819.0,16.52056,Pré-saturé,2294,Av_Champs_Elysees-Washington,2293,Av_Champs_Elysees-Berri,Invalide,2005-01-01,2019-06-01,"48.87153587897717, 2.3017227924560615","{""coordinates"": [[2.3009951475338766, 48.87177..."
2,4264,AV_Champs_Elysees,2023-02-01T20:00:00+01:00,1082.0,19.98722,Pré-saturé,2294,Av_Champs_Elysees-Washington,2293,Av_Champs_Elysees-Berri,Invalide,2005-01-01,2019-06-01,"48.87153587897717, 2.3017227924560615","{""coordinates"": [[2.3009951475338766, 48.87177..."
3,4264,AV_Champs_Elysees,2023-02-01T18:00:00+01:00,1027.0,23.27334,Pré-saturé,2294,Av_Champs_Elysees-Washington,2293,Av_Champs_Elysees-Berri,Invalide,2005-01-01,2019-06-01,"48.87153587897717, 2.3017227924560615","{""coordinates"": [[2.3009951475338766, 48.87177..."
4,4264,AV_Champs_Elysees,2022-12-07T00:00:00+01:00,528.0,68.14334,Bloqué,2294,Av_Champs_Elysees-Washington,2293,Av_Champs_Elysees-Berri,Invalide,2005-01-01,2019-06-01,"48.871535879911605, 2.3017227924560624","{""coordinates"": [[2.300995147533877, 48.871777..."


In [3]:
champ['time']=pd.to_datetime(champ['Date et heure de comptage'],utc=True)
champ['debit']=champ['Débit horaire']
champ['taux']=champ['Taux d\'occupation']
champ=champ[['time','debit','taux']]
champ = champ.sort_values(by='time')
champ.set_index('time', inplace=True)


In [32]:
champ.isna().sum()

time       0
debit    267
taux     288
dtype: int64

### benchmark simplest model: ignore missing values/ estimate the debit and taux for each weekday as the average of the last same 4 weekdays    

In [78]:
# Convert 'time' column to datetime if it's not already
champ['time'] = pd.to_datetime(champ['time'])

# Set 'time' column as the index
champ.set_index('time', inplace=True)


last_10_days = champ.last('10D')

def calculate_avg_last_4_weeks(df, date, column):
    day_of_week = date.dayofweek
    hour = date.hour
    # Filter for the same day of the week in the last 4 weeks
    past_data = df[(df.index < date) & 
                   (df.index > date - pd.Timedelta(weeks=4)) & 
                   (df.index.dayofweek == day_of_week) & 
                   (df.index.hour == hour)][column]

    # Get the last 4 values and calculate the mean, ignoring NaNs
    return past_data.tail(4).mean(skipna=True)

for date, row in last_10_days.iterrows():
    last_10_days.at[date, 'taux_pred'] = calculate_avg_last_4_weeks(champ, date, 'taux')
    last_10_days.at[date, 'debit_pred'] = calculate_avg_last_4_weeks(champ, date, 'debit')



C:\Users\alkhamed\AppData\Local\Temp\ipykernel_736\3623180710.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_10_days.at[date, 'taux_pred'] = calculate_avg_last_4_weeks(champ, date, 'taux')
C:\Users\alkhamed\AppData\Local\Temp\ipykernel_736\3623180710.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_10_days.at[date, 'debit_pred'] = calculate_avg_last_4_weeks(champ, date, 'debit')


In [80]:
from sklearn.metrics import mean_squared_error
mse_taux = mean_squared_error(last_10_days['taux'], last_10_days['taux_pred'])

mse_debit = mean_squared_error(last_10_days['debit'], last_10_days['debit_pred'])


print(f'MSE_taux : {mse_taux}')
print(f'MSE_debit : {mse_debit}')


MSE_taux : 279.62497252318786
MSE_debit : 60898.26932870371


In [4]:
# Assuming 'champ' is indexed by datetime
last_10_days = champ.last('10D')
# Function to get the value from the same weekday and hour in the previous week
def get_previous_week_value(df, current_time, column):
    # Calculate the datetime for the same weekday and hour in the previous week
    time_last_week = current_time - pd.Timedelta(weeks=1)

    # Return the value if it exists, else NaN
    return df.loc[time_last_week, column] if time_last_week in df.index else float('nan')

# Adding the new columns
last_10_days['taux_last_week'] = [get_previous_week_value(champ, idx, 'taux') for idx in last_10_days.index]
last_10_days['debit_last_week'] = [get_previous_week_value(champ, idx, 'debit') for idx in last_10_days.index]


C:\Users\alkhamed\AppData\Local\Temp\ipykernel_27880\730357461.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_10_days['taux_last_week'] = [get_previous_week_value(champ, idx, 'taux') for idx in last_10_days.index]
C:\Users\alkhamed\AppData\Local\Temp\ipykernel_27880\730357461.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_10_days['debit_last_week'] = [get_previous_week_value(champ, idx, 'debit') for idx in last_10_days.index]


In [8]:
last_10_days.head()

,debit,taux,taux_last_week,debit_last_week
time,,,,
2023-11-22 00:00:00+00:00,645.0,8.48500,20.30778,852.0
2023-11-22 01:00:00+00:00,546.0,7.33056,42.79500,905.0
2023-11-22 02:00:00+00:00,407.0,5.82500,28.79945,747.0
2023-11-22 03:00:00+00:00,353.0,5.46445,37.76333,574.0
2023-11-22 04:00:00+00:00,387.0,9.54945,34.72278,647.0


In [15]:
# Filter data for 15th of November
data_nov_15 = champ[(champ.index.month == 11) & (champ.index.day == 15)&(champ.index.year==2023)]

# Print the data
print(data_nov_15)



                            debit      taux
time                                       
2023-11-15 00:00:00+00:00   852.0  20.30778
2023-11-15 01:00:00+00:00   905.0  42.79500
2023-11-15 02:00:00+00:00   747.0  28.79945
2023-11-15 03:00:00+00:00   574.0  37.76333
2023-11-15 04:00:00+00:00   647.0  34.72278
2023-11-15 05:00:00+00:00   669.0  23.08222
2023-11-15 06:00:00+00:00  1122.0  20.54611
2023-11-15 07:00:00+00:00  1501.0  32.66723
2023-11-15 08:00:00+00:00  1659.0  45.52444
2023-11-15 09:00:00+00:00  1588.0  34.66723
2023-11-15 10:00:00+00:00  1568.0  37.51278
2023-11-15 11:00:00+00:00  1383.0  54.34222
2023-11-15 12:00:00+00:00  1285.0  48.25667
2023-11-15 13:00:00+00:00  1326.0  56.54556
2023-11-15 14:00:00+00:00  1403.0  34.16334
2023-11-15 15:00:00+00:00  1328.0  26.15389
2023-11-15 16:00:00+00:00  1537.0  35.84445
2023-11-15 17:00:00+00:00  1536.0  35.83778
2023-11-15 18:00:00+00:00  1431.0  28.16278
2023-11-15 19:00:00+00:00  1550.0  28.69667
2023-11-15 20:00:00+00:00  1578.

### naive benchmark taking debit and taux as the values of the previous week in the same time

In [16]:
from sklearn.metrics import mean_squared_error

filtered_data = last_10_days.dropna(subset=['debit', 'debit_last_week'])
mse = mean_squared_error(filtered_data['debit'], filtered_data['debit_last_week'])
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 82224.07916666666


In [17]:
filtered_data = last_10_days.dropna(subset=['taux', 'taux_last_week'])
mse = mean_squared_error(filtered_data['taux'], filtered_data['taux_last_week'])
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 347.0460493226865


In [19]:
last_10_days['taux_variation'] = ((last_10_days['taux'] - last_10_days['taux_last_week']) / last_10_days['taux_last_week']).abs()
last_10_days['debit_variation'] = ((last_10_days['debit'] - last_10_days['debit_last_week']) / last_10_days['debit_last_week']).abs()
# Filter rows where variation is greater than 20%
significant_variation = last_10_days[(last_10_days['taux_variation'] > 0.2) | (last_10_days['debit_variation'] > 0.2)]

# Print the filtered rows
significant_variation.shape


C:\Users\alkhamed\AppData\Local\Temp\ipykernel_27880\1612248704.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_10_days['taux_variation'] = ((last_10_days['taux'] - last_10_days['taux_last_week']) / last_10_days['taux_last_week']).abs()
C:\Users\alkhamed\AppData\Local\Temp\ipykernel_27880\1612248704.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_10_days['debit_variation'] = ((last_10_days['debit'] - last_10_days['debit_last_week']) / last_10_days['debit_last_week']).abs()


(177, 6)

In [21]:
significant_variation.head(50)

,debit,taux,taux_last_week,debit_last_week,taux_variation,debit_variation
time,,,,,,
2023-11-22 00:00:00+00:00,645.0,8.48500,20.30778,852.0,0.582180,0.242958
2023-11-22 01:00:00+00:00,546.0,7.33056,42.79500,905.0,0.828705,0.396685
2023-11-22 02:00:00+00:00,407.0,5.82500,28.79945,747.0,0.797739,0.455154
2023-11-22 03:00:00+00:00,353.0,5.46445,37.76333,574.0,0.855297,0.385017
2023-11-22 04:00:00+00:00,387.0,9.54945,34.72278,647.0,0.724980,0.401855
2023-11-22 05:00:00+00:00,610.0,12.95334,23.08222,669.0,0.438817,0.088191
2023-11-22 06:00:00+00:00,853.0,17.78500,20.54611,1122.0,0.134386,0.239750
2023-11-22 07:00:00+00:00,1209.0,15.42722,32.66723,1501.0,0.527746,0.194537
2023-11-22 08:00:00+00:00,1488.0,16.22000,45.52444,1659.0,0.643708,0.103074


In [25]:
data_nov_15 = champ[(champ.index.month == 11) & (champ.index.day == 15)&(champ.index.year==2022)]

# Print the data
print(data_nov_15)

                            debit      taux
time                                       
2022-11-15 00:00:00+00:00   526.0   7.36778
2022-11-15 01:00:00+00:00   379.0   5.00056
2022-11-15 02:00:00+00:00   291.0   4.33611
2022-11-15 03:00:00+00:00   192.0   2.35722
2022-11-15 04:00:00+00:00   185.0   2.38222
2022-11-15 05:00:00+00:00   364.0   4.58278
2022-11-15 06:00:00+00:00   641.0   6.97556
2022-11-15 07:00:00+00:00  1032.0  11.75778
2022-11-15 08:00:00+00:00  1284.0  14.43333
2022-11-15 09:00:00+00:00  1292.0  16.94555
2022-11-15 10:00:00+00:00  1198.0  18.15334
2022-11-15 11:00:00+00:00  1300.0  23.86334
2022-11-15 12:00:00+00:00  1262.0  25.75111
2022-11-15 13:00:00+00:00  1198.0  27.95278
2022-11-15 14:00:00+00:00  1192.0  22.09000
2022-11-15 15:00:00+00:00  1218.0  22.68889
2022-11-15 16:00:00+00:00  1128.0  19.58278
2022-11-15 17:00:00+00:00  1140.0  19.75833
2022-11-15 18:00:00+00:00  1117.0  17.14889
2022-11-15 19:00:00+00:00  1092.0  17.26278
2022-11-15 20:00:00+00:00  1015.